In [7]:
import numpy as np
from nltk.tokenize import word_tokenize
import nltk
import json 
import os
import pickle

def loadData():

    with open('quora_raw_train.json') as f:
        training_examples = json.load(f)
        
    with open('quora_raw_val.json') as f:
        val_examples = json.load(f)

    with open('quora_data_prepro.json') as f:
        vocab = json.load(f)['ix_to_word'] # Vocab = {1: 'the', 2: 'sauce' .....}
        vocab_inv = {word:number for number,word in list(vocab.items())} # Inverted dictionary created          Vocab_inv = {'the':1, 'sauce':2, .....}

    X_train_prelim = [training_examples[i]['question1'] for i in range(len(training_examples))] # Training examples before preprocessing
    Y_train_prelim = [training_examples[i]['question'] for i in range(len(training_examples))]

    X_val_prelim = [val_examples[i]['question1'] for i in range(len(val_examples))]
    Y_val_prelim = [val_examples[i]['question'] for i in range(len(val_examples))]

    X_train = []
    Y_train = []

    filename = "glove.6B.100d.txt"

    lines = open(filename).readlines()
    print(len(lines))
    embeddings = {}
    a = 0
    for line in lines:
        a +=1
        print(a)
        word = line.split()[0]
        embedding = list(map(float, line.split()[1:]))
        if word in vocab.values():
            embeddings[int(vocab_inv[word])] = embedding # This is creating a dictionary with indexes corresponding to the position of the particular word in the vocabulary
            # embeddings = {1:embedding of 'the', 2: embedding of 'sauce'....}
        
    embedded_words = embeddings
    
# This is basically tokenising each sentence, and converting it into a list of word embeddings. X corresponds to a training example and Y corresponds to its paraphrase
    for i in range(len(X_train_prelim)): 
        print(i)
        sentence_x = X_train_prelim[i]
        sentence_y = Y_train_prelim[i]
        tokensx = word_tokenize(sentence_x)
        tokensy = word_tokenize(sentence_y)
        vectorx = []
        vectory = []
        for word in tokensx:
            try:
                vectorx.append(embedded_words[int(vocab_inv[word])])
            except:
#If the word is not in the embeddings given, then add a random vector (might have to do something about this :) ) (Potentially could write a spell checker to find misspellings)
                vectorx.append(-2*torch.rand(100,1)) #
        for word in tokensy:
            try:
                vectorx.append(embedded_words[int(vocab_inv[word])])
            except:
                vectory.append(-2*torch.rand(100,1))

        X_train.append(vectorx)
        Y_train.append(vectory)

    return embedded_words

In [8]:
from Decoder import Decoder
from Encoder import Encoder

input_size = 100
hidden_size_enc = 128
hidden_size_dec = 20
num_layers = 1
vocab_size = 5 #len(vocab)
dropout = 0.2

embedded_words = loadData()

encoder = Encoder(input_size,hidden_size_enc,dropout,vocab_size,num_layers, embedded_words)
decoder = Decoder(encoder, input_dims, hidden_size_dec)

FileNotFoundError: [Errno 2] No such file or directory: 'quora_raw_val.json'